##Capstone project Data Science 

In [1]:
%matplotlib inline

import matplotlib as mpl
import matplotlib.path as mpltPath
import pandas as pd
import numpy as np

import folium
from folium import plugins

print('Folium installed and imported!')

print("Hello Capstone Project Course!")

Folium installed and imported!
Hello Capstone Project Course!


In [2]:
crash_data = pd.read_csv('https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv')

print('Data downloaded and read into a dataframe!')

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Data downloaded and read into a dataframe!


In [3]:
#Change Collisontype and a number of fields into numerical data so correlations can be drawn
#Dummy variables

crash_data.isnull().sum()

SEVERITYCODE           0
X                   5334
Y                   5334
OBJECTID               0
INCKEY                 0
COLDETKEY              0
REPORTNO               0
STATUS                 0
ADDRTYPE            1926
INTKEY            129603
LOCATION            2677
EXCEPTRSNCODE     109862
EXCEPTRSNDESC     189035
SEVERITYCODE.1         0
SEVERITYDESC           0
COLLISIONTYPE       4904
PERSONCOUNT            0
PEDCOUNT               0
PEDCYLCOUNT            0
VEHCOUNT               0
INCDATE                0
INCDTTM                0
JUNCTIONTYPE        6329
SDOT_COLCODE           0
SDOT_COLDESC           0
INATTENTIONIND    164868
UNDERINFL           4884
WEATHER             5081
ROADCOND            5012
LIGHTCOND           5170
PEDROWNOTGRNT     190006
SDOTCOLNUM         79737
SPEEDING          185340
ST_COLCODE            18
ST_COLDESC          4904
SEGLANEKEY             0
CROSSWALKKEY           0
HITPARKEDCAR           0
dtype: int64

In [4]:
# clean up the dataset to remove unnecessary columns (eg. REG) 
#crash_data.drop(['INCKEY','OBJECTID','COLDETKEY','SDOTCOLNUM'], axis=1, inplace=True)


# for sake of consistency, let's also make all column labels of type string
crash_data.columns = list(map(str, crash_data.columns))

#Remove Missing Lightcondition
# simply drop whole row with NaN in "price" column
crash_data.dropna(subset=["LIGHTCOND"], axis=0, inplace=True)

# reset index, because we droped two rows
crash_data.reset_index(drop=True, inplace=True)

crash_data.dtypes


SEVERITYCODE        int64
X                 float64
Y                 float64
OBJECTID            int64
INCKEY              int64
COLDETKEY           int64
REPORTNO           object
STATUS             object
ADDRTYPE           object
INTKEY            float64
LOCATION           object
EXCEPTRSNCODE      object
EXCEPTRSNDESC      object
SEVERITYCODE.1      int64
SEVERITYDESC       object
COLLISIONTYPE      object
PERSONCOUNT         int64
PEDCOUNT            int64
PEDCYLCOUNT         int64
VEHCOUNT            int64
INCDATE            object
INCDTTM            object
JUNCTIONTYPE       object
SDOT_COLCODE        int64
SDOT_COLDESC       object
INATTENTIONIND     object
UNDERINFL          object
WEATHER            object
ROADCOND           object
LIGHTCOND          object
PEDROWNOTGRNT      object
SDOTCOLNUM        float64
SPEEDING           object
ST_COLCODE         object
ST_COLDESC         object
SEGLANEKEY          int64
CROSSWALKKEY        int64
HITPARKEDCAR       object
dtype: objec

In [5]:
#Correlations without data wrangling

#Dummy variables for two things
crash_df = pd.DataFrame(crash_data, columns = ['ADDRTYPE', 'COLLISIONTYPE'])

dummies = pd.get_dummies(crash_df["ADDRTYPE"])
dummies2 = pd.get_dummies(crash_df["COLLISIONTYPE"])
crash_df = pd.concat([crash_df, dummies], axis=1)
crash_df = pd.concat([crash_df, dummies2], axis=1)

#Add the Severity code to this
crash_df = pd.concat([crash_df, crash_data["SEVERITYCODE"]], axis=1)

#Correlate the data
crash_df.corr()

#Blocks, Parked Cars, and Sideswipes are most correlated with property damage
#Intersections, Cycles, and Rear-ends are most correlated with injury collisions

,Alley,Block,Intersection,Angles,Cycles,Head On,Left Turn,Other,Parked Car,Pedestrian,Rear Ended,Right Turn,Sideswipe,SEVERITYCODE
Alley,1.000000,-0.085877,-0.044600,-0.017274,-0.006717,-0.002413,-0.017534,0.049225,0.026556,0.005086,-0.026990,-0.007904,-0.016180,-0.026552
Block,-0.085877,1.000000,-0.970433,-0.484351,-0.081291,0.027271,-0.291048,0.135314,0.354953,-0.147477,0.214220,-0.062279,0.090510,-0.186691
Intersection,-0.044600,-0.970433,1.000000,0.498942,0.086164,-0.025760,0.300600,-0.141354,-0.379300,0.151818,-0.210298,0.065028,-0.087653,0.201167
Angles,-0.017274,-0.484351,0.498942,1.000000,-0.081065,-0.049077,-0.131989,-0.178646,-0.274923,-0.089830,-0.221363,-0.059501,-0.155948,0.094415
Cycles,-0.006717,-0.081291,0.086164,-0.081065,1.000000,-0.017788,-0.047839,-0.064750,-0.099646,-0.032559,-0.080233,-0.021566,-0.056523,0.214749
Head On,-0.002413,0.027271,-0.025760,-0.049077,-0.017788,1.000000,-0.028962,-0.039200,-0.060326,-0.019711,-0.048574,-0.013056,-0.034219,0.029312
Left Turn,-0.017534,-0.291048,0.300600,-0.131989,-0.047839,-0.028962,1.000000,-0.105425,-0.162242,-0.053012,-0.130635,-0.035114,-0.092031,0.057031
Other,0.049225,0.135314,-0.141354,-0.178646,-0.064750,-0.039200,-0.105425,1.000000,-0.219594,-0.071751,-0.176813,-0.047526,-0.124563,-0.035732
Parked Car,0.026556,0.354953,-0.379300,-0.274923,-0.099646,-0.060326,-0.162242,-0.219594,1.000000,-0.110420,-0.272103,-0.073140,-0.191693,-0.311365
Pedestrian,0.005086,-0.147477,0.151818,-0.089830,-0.032559,-0.019711,-0.053012,-0.071751,-0.110420,1.000000,-0.088908,-0.023898,-0.062635,0.247090


In [6]:

#Look at different value set
crash_type = pd.DataFrame(crash_data, columns = ['SEVERITYCODE','PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT'])

#If more person than vehicles, that means either pedestrian or multiple people in a car
#Only 1 vehicle or 2
dummies = pd.get_dummies(crash_type['VEHCOUNT'])
crash_type = pd.concat([crash_type, dummies], axis=1)

crash_type.corr()


,SEVERITYCODE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,0,1,2,3,4,5,6,7,8,9,10,11,12
SEVERITYCODE,1.000000,0.129730,0.248004,0.215716,-0.080742,0.048308,0.212983,-0.235405,0.070601,0.035451,0.022149,0.006636,0.006010,0.000621,0.005488,0.004947,0.002437,-0.001509
PERSONCOUNT,0.129730,1.000000,-0.025639,-0.040938,0.400699,-0.024570,-0.239016,-0.018752,0.234629,0.182864,0.123166,0.084047,0.058702,0.040749,0.030433,0.022869,0.033383,0.016171
PEDCOUNT,0.248004,-0.025639,1.000000,-0.017900,-0.316323,0.059621,0.451427,-0.341488,-0.047353,-0.019269,-0.005562,-0.004326,-0.001271,-0.001690,-0.001309,-0.000617,-0.001069,-0.000436
PEDCYLCOUNT,0.215716,-0.040938,-0.017900,1.000000,-0.306056,0.198230,0.407079,-0.315819,-0.046250,-0.019556,-0.009093,-0.004772,-0.002678,-0.001529,-0.001184,-0.000558,-0.000967,-0.000395
VEHCOUNT,-0.080742,0.400699,-0.316323,-0.306056,1.000000,-0.114527,-0.689164,0.099940,0.500788,0.414065,0.287399,0.200616,0.140514,0.096184,0.086859,0.046769,0.091096,0.041308
0,0.048308,-0.024570,0.059621,0.198230,-0.114527,1.000000,-0.012845,-0.060694,-0.008794,-0.003689,-0.001715,-0.000900,-0.000505,-0.000288,-0.000223,-0.000105,-0.000182,-0.000074
1,0.212983,-0.239016,0.451427,0.407079,-0.689164,-0.012845,1.000000,-0.741654,-0.107465,-0.045079,-0.020962,-0.011001,-0.006173,-0.003525,-0.002730,-0.001287,-0.002229,-0.000910
2,-0.235405,-0.018752,-0.341488,-0.315819,0.099940,-0.060694,-0.741654,1.000000,-0.507762,-0.212995,-0.099043,-0.051980,-0.029169,-0.016655,-0.012901,-0.006081,-0.010533,-0.004300
3,0.070601,0.234629,-0.047353,-0.046250,0.500788,-0.008794,-0.107465,-0.507762,1.000000,-0.030863,-0.014351,-0.007532,-0.004227,-0.002413,-0.001869,-0.000881,-0.001526,-0.000623
4,0.035451,0.182864,-0.019269,-0.019556,0.414065,-0.003689,-0.045079,-0.212995,-0.030863,1.000000,-0.006020,-0.003159,-0.001773,-0.001012,-0.000784,-0.000370,-0.000640,-0.000261


In [7]:
#Light conditions
crash_day = pd.DataFrame(crash_data, columns = ['SEVERITYCODE','LIGHTCOND'])

dummies = pd.get_dummies(crash_day['LIGHTCOND'])
crash_type = pd.concat([crash_day, dummies], axis=1)

crash_type.corr()


,SEVERITYCODE,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dark - Unknown Lighting,Dawn,Daylight,Dusk,Other,Unknown
SEVERITYCODE,1.000000,-0.016555,-0.006565,-0.003699,0.001035,0.007067,0.083855,0.010971,-0.006146,-0.154600
Dark - No Street Lights,-0.016555,1.000000,-0.007216,-0.053039,-0.000689,-0.010460,-0.113772,-0.016213,-0.003186,-0.025017
Dark - Street Lights Off,-0.006565,-0.007216,1.000000,-0.046804,-0.000608,-0.009230,-0.100396,-0.014307,-0.002812,-0.022076
Dark - Street Lights On,-0.003699,-0.053039,-0.046804,1.000000,-0.004469,-0.067845,-0.737967,-0.105162,-0.020668,-0.162270
Dark - Unknown Lighting,0.001035,-0.000689,-0.000608,-0.004469,1.000000,-0.000881,-0.009586,-0.001366,-0.000268,-0.002108
Dawn,0.007067,-0.010460,-0.009230,-0.067845,-0.000881,1.000000,-0.145532,-0.020739,-0.004076,-0.032001
Daylight,0.083855,-0.113772,-0.100396,-0.737967,-0.009586,-0.145532,1.000000,-0.225580,-0.044334,-0.348078
Dusk,0.010971,-0.016213,-0.014307,-0.105162,-0.001366,-0.020739,-0.225580,1.000000,-0.006318,-0.049602
Other,-0.006146,-0.003186,-0.002812,-0.020668,-0.000268,-0.004076,-0.044334,-0.006318,1.000000,-0.009748
Unknown,-0.154600,-0.025017,-0.022076,-0.162270,-0.002108,-0.032001,-0.348078,-0.049602,-0.009748,1.000000


In [8]:
#Road and Weather conditions
crash_road = pd.DataFrame(crash_data, columns = ['SEVERITYCODE','WEATHER','ROADCOND'])

dummies = pd.get_dummies(crash_road['ROADCOND'])
crash_road = pd.concat([crash_road, dummies], axis=1)

dummies2 = pd.get_dummies(crash_road['WEATHER'])
crash_road = pd.concat([crash_road, dummies2], axis=1)

crash_road.corr()


,SEVERITYCODE,Dry,Ice,Oil,Other,Sand/Mud/Dirt,Snow/Slush,Standing Water,Unknown,Wet,...,Clear,Fog/Smog/Smoke,Other,Overcast,Partly Cloudy,Raining,Severe Crosswind,Sleet/Hail/Freezing Rain,Snowing,Unknown
SEVERITYCODE,1.000000,0.061925,-0.013147,0.002952,0.001544,0.000410,-0.021320,-0.002172,-0.161035,0.038569,...,0.055154,0.003270,-0.023198,0.012925,0.003344,0.036053,-0.000533,-0.002849,-0.017133,-0.158310
Dry,0.061925,1.000000,-0.110683,-0.025399,-0.036345,-0.027312,-0.100649,-0.034052,-0.405747,-0.798522,...,0.748284,-0.003135,-0.059893,-0.063193,0.001554,-0.617259,-0.006199,-0.031023,-0.092281,-0.371422
Ice,-0.013147,-0.110683,1.000000,-0.001472,-0.002107,-0.001583,-0.005834,-0.001974,-0.023518,-0.046284,...,-0.008851,0.038019,0.013824,0.009098,-0.000411,-0.034417,0.016399,0.046933,0.157361,-0.005374
Oil,0.002952,-0.025399,-0.001472,1.000000,-0.000483,-0.000363,-0.001339,-0.000453,-0.005397,-0.010621,...,-0.010785,-0.001009,0.003143,-0.001100,-0.000094,0.016496,-0.000211,-0.000449,-0.001270,-0.002211
Other,0.001544,-0.036345,-0.002107,-0.000483,1.000000,-0.000520,-0.001916,-0.000648,-0.007723,-0.015198,...,-0.004790,0.005896,0.028737,0.004465,-0.000135,-0.007346,-0.000302,0.024031,0.001101,0.001930
Sand/Mud/Dirt,0.000410,-0.027312,-0.001583,-0.000363,-0.000520,1.000000,-0.001440,-0.000487,-0.005803,-0.011421,...,-0.005615,0.008679,0.002747,0.004677,-0.000102,-0.001360,-0.000227,-0.000483,0.006399,0.002099
Snow/Slush,-0.021320,-0.100649,-0.005834,-0.001339,-0.001916,-0.001440,1.000000,-0.001795,-0.021386,-0.042088,...,-0.072734,-0.001335,0.002920,-0.002708,-0.000374,-0.015876,-0.000837,0.078778,0.574139,0.003395
Standing Water,-0.002172,-0.034052,-0.001974,-0.000453,-0.000648,-0.000487,-0.001795,1.000000,-0.007235,-0.014239,...,-0.026265,-0.001352,-0.001630,-0.005948,-0.000127,0.040003,-0.000283,0.025731,-0.001703,-0.001689
Unknown,-0.161035,-0.405747,-0.023518,-0.005397,-0.007723,-0.005803,-0.021386,-0.007235,1.000000,-0.169671,...,-0.329036,-0.013616,0.138813,-0.101907,-0.001508,-0.133175,-0.003373,-0.007172,-0.018024,0.894182
Wet,0.038569,-0.798522,-0.046284,-0.010621,-0.015198,-0.011421,-0.042088,-0.014239,-0.169671,1.000000,...,-0.598376,0.004578,-0.026215,0.131442,-0.000597,0.766552,0.006090,0.013828,-0.012669,-0.150463


In [10]:
#Look at Col_code to see the types of things that correlate with injury collision, and then visualize the number of injury collisions with col_code
Colcode = crash_data["SDOT_COLCODE"].value_counts(sort=True)

Colcode
#If a col_code corresponds to a certain area, we can say it predicts it if X and Y Severe_Crash. Basically, if X and Y match? 
#0, 51, 56 are most common

11    83195
14    52687
16     9796
0      8944
28     8764
24     6383
13     5775
26     4645
18     3033
15     1571
12     1423
51     1292
29      473
21      180
56      178
27      161
54      134
23      123
48      106
31      103
25      101
34       92
64       74
69       68
33       53
55       50
66       23
22       16
32       12
53        9
44        8
61        7
35        6
68        4
36        4
58        4
46        3
52        2
47        1
Name: SDOT_COLCODE, dtype: int64

In [14]:
#Explore INCDATE to see if these things have changed over time, gotten worse, etc. 
import datetime
import seaborn as sns

bins = np.linspace
crash_data["INCDATE"].plot(kind='bar',figsize=(14,8))


df_top5.index = df_top5.index.map(int) # let's change the index values of df_top5 to type integer for plotting
df_top5.plot(kind='area', 
             stacked=False,
             figsize=(20, 10), # pass a tuple (x, y) size
             )

plt.title('Immigration Trend of Top 5 Countries')
plt.ylabel('Number of Immigrants')
plt.xlabel('Years')

plt.show()

TypeError: no numeric data to plot

In [ ]:
#After finding the predictive variable, start working on ML (SVM?) For predictive model. 

In [ ]:
#We use limits to take a small sample of the data and see how different variables compare. From there, apply test/training data models